In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf

In [ ]:
#reading in county/year level data

df = pd.read_excel('HPI_AT_BDL_county (1).xlsx', header=6)

In [ ]:
#selecting my county as an example

df = df[df['FIPS code']==34013]
df = df.set_index('Year')['HPI']
df = df.apply(lambda x: float(x))
df.index = pd.to_datetime(df.index, format='%Y')
df.columns = ['HPI']

df.plot()

In [ ]:
#seasonal decomposition- no seasonal trend since this is yearly data
#looks like all that exists is the trend
result = seasonal_decompose(df, model='multiplicative')
fig = result.plot()

In [ ]:
#function from Medium article
#used to test whether the time series is stationary

def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=12).mean()
    rolstd = timeseries.rolling(window=12).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
#not stationary

test_stationarity(df)

In [ ]:
#reading in montly/regional HPI for a montly time series example

data = pd.read_excel('HPI_PO_monthly_hist.xls', header=3)
data.head()

In [ ]:
data = data[['Month', 'East North Central\n(NSA)']]
data = data.iloc[1:, :]
data = data.set_index('Month')
data.index = pd.to_datetime(data.index)
data.columns = ['east_north_central_nsa']
data['east_north_central_nsa'] = data['east_north_central_nsa'].apply(lambda x: float(x))
data.plot()

In [ ]:
#seasonal decomposition (seasonal trend applies here)
result = seasonal_decompose(data, model='multiplicative')
fig = result.plot()

In [ ]:
#not stationary
test_stationarity(data)

In [ ]:
#take log
data_log = np.log(data)
data_log.plot()

In [ ]:
#rolling mean
moving_avg = data_log.rolling(12).mean()
plt.plot(data_log)
plt.plot(moving_avg,color='red')

In [ ]:
#differencing
data_log_moving_avg_diff = data_log - moving_avg
data_log_moving_avg_diff.head(20)

In [ ]:
#drop na due to differencing (first 12 values)
data_log_moving_avg_diff.dropna(inplace=True)
data_log_moving_avg_diff.head()

In [ ]:
#not stationary but closer (look at p-value)
test_stationarity(data_log_moving_avg_diff)

In [ ]:
#exponential weighted average
expweighted_avg = data_log.ewm(halflife=12).mean()
plt.plot(data_log)
plt.plot(expweighted_avg, color='red')

In [ ]:
#differencing with ewma- still not stationary
data_log_ewma_diff = data_log - expweighted_avg
test_stationarity(data_log_ewma_diff)

In [ ]:
#taking shift for differencing
data_log_diff = data_log - data_log.shift()
plt.plot(data_log_diff)

In [ ]:
#not stationary
data_log_diff.dropna(inplace=True)
test_stationarity(data_log_diff)

In [ ]:
#decomposition of log transform
decomposition = seasonal_decompose(data_log)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(data_log, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal, label='Seasonal')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residual')
plt.legend(loc='best')

In [ ]:
#the residual of the log-transform decomposition is stationary
data_log_decompose = residual
data_log_decompose.dropna(inplace=True)
test_stationarity(data_log_decompose)

In [ ]:
#using autocorrelation and partial autocorrelation to find p and q values for ARIMA
#where PACF first crosses upper CI is p (looks like 2)
#where ACF first crosses upper CI is q (looks like 3)

lag_acf = acf(data_log_diff, nlags=20)
lag_pacf = pacf(data_log_diff, nlags=20, method='ols')

#Plot ACF
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(data_log_diff)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(data_log_diff)), linestyle='--', color='gray')
plt.title('Autocorrelation Function')

#Plot PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(data_log_diff)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(data_log_diff)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function')

plt.tight_layout()

In [ ]:
#p=2, q=3

#AR model
model = ARIMA(data_log, order=(2, 1, 0))
results_AR = model.fit(disp=-1)
plt.plot(data_log_diff)
plt.plot(results_AR.fittedvalues, color='red')

AR = pd.DataFrame(results_AR.fittedvalues)
AR.columns = ['east_north_central_nsa']
plt.title('RSS: %.4f' % ((AR - data_log_diff)**2).sum())

In [ ]:
#p=2, q=3

#MA model
model = ARIMA(data_log, order=(0, 1, 3))
results_MA = model.fit(disp=-1)
plt.plot(data_log_diff)
plt.plot(results_MA.fittedvalues, color='red')

MA = pd.DataFrame(results_MA.fittedvalues)
MA.columns = ['east_north_central_nsa']
plt.title('RSS: %.4f' % ((MA - data_log_diff)**2).sum())

In [ ]:
#p=2, q=3

#ARIMA model
model = ARIMA(data_log, order=(2, 1, 3))
results_ARIMA = model.fit(disp=-1)
plt.plot(data_log_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')

ARIMA = pd.DataFrame(results_ARIMA.fittedvalues)
ARIMA.columns = ['east_north_central_nsa']
plt.title('RSS: %.4f' % ((ARIMA - data_log_diff)**2).sum())

In [ ]:
#ARIMA predictions series
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions_ARIMA_diff.head()

In [ ]:
#transforming back to original form- remove differencing
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_diff_cumsum.head()

In [ ]:
#transforming back to original form- add back from start value of log_transform
start_value = data_log.iloc[0][0]
predictions_ARIMA_log = predictions_ARIMA_diff_cumsum.apply(lambda x: x+start_value)
predictions_ARIMA_log.head()

In [ ]:
#undo log transform and plot actual (blue) vs ARIMA predictions (orange)
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(data)
plt.plot(predictions_ARIMA)